# Bazy Danych 2

## Databases in-memory

### Wstęp
Bazy danych "in-memory", czyli bazy danych przechowujące swoje dane w pamięci operacyjnej (najczęściej RAM). 
W przeciwieństwie do tradycyjnych baz danych takich jak np. SQL server czy Oracle, które domyślnie przechowują swoje dane w pamięci trwałej (lecz również posiadają mechanizmy in-memory).

Istnieją bazy gdzie wszystkie dane są trzymane w pamięci operacyjnej, ale również rozwiązania hybrydowe gdzie tylko część z nich jest tak przechowywana.

#### Plusy
Trzymanie danych w pamięci RAM pozwala na znacznie krótsze czasy dostępu w porównaniu do danych trzymanych na dysku.
W przypadku dysków obrotowych eliminowany jest tak zwany "seek time" czyli czas wykorzystany przez głowice na fizyczne przemieszczenie po dysku.

Dla przykładu według tego [źródła](https://gist.github.com/jboner/2841832):
Czas potrzebny na przeczytanie 1MB z pamięci wynosi: 150 mikrosekund

gdzie czas potrzebny na przeczytanie 1MB z dysku twardego wynosi 20,000 mikrosekund, jest to aż 80 razy więcej!

#### Problemy

Jednak istnieje jeden problem związany z przechowywaniem danych w pamięci RAM, który jest ulotna. Gdy wyłączymy zasilanie, nasze dane będą utracone.

Dlatego więc nie powinno wykorzystywać się baz tego typu do przechowywania ważnych danych, istnieją jednak sytuacje gdzie takie bazy są optymalnym narzędziem.

Istnieją jednak mechanizmy hybrydowe pozwalające zaradzić temu problemu, np poprzez tworzenie kopii danych na dysku co jakiś czas (snapshot), zapisywanie logów transakcji (aby później odtworzyć taką bazę według tego pliku), czy nawet rozwiązania jak używanie nieulotnej pamięci operacyjnej (NVDIMM).

#### Przykładowe bazy

Przykładowymi bazami "in-memory", są np. Redis czy Memcached, są one open-source i używane w wielu firmach, instytucjach. Postaram się je pokazać w tej prezentacji przy użyciu języka Python.

---

### Memcached

Przedstawię użycie przykładowej bazy in-memory "Memcached". Jest to baza powszechnie używana do tworzenia tzw. "cache" czyli zapamiętywania wartości zwrotnych np. odpowiedzi na requesty typu Get.

Jest ona rozproszona i wielowątkowa, przez to też bardzo dobrze skalowalna poziomowo nawet do skali takich jak Google, Twitter, Wikipedia...


Działa w architekturze klient-serwer, uruchamiany jest serwer z mapą typu key-value, gdzie klucze mają wielkość max 1mb oraz klucze/wartości muszą być bajtami.

Przykładowo 1000 klientów pyta nas o średnią temperaturę w Krakowie w dniu 25.01.2023, możemy albo pytać 1000 razy API pogodowe, lub zapisać taką odpowiedź do Memcached i zwracać ją zapisaną w bazie, końcowo dla X klientów użyjemy i tak tylko 1 requesta, zapobiegając np. przekroczeniu limitu API. 

Jest to tak zwane użycie bazy jako cache. Gdzie zapisujemy dla jakichś kluczy wartość, aby móc szybko ją odzyskać (przy pomocy np. hashowania).

#### Użycie

Użyję języka Python, lecz taka baza może być używana z innymi językami.

Potrzebujemy ją instalować, na Ubuntu możemy to zrobić poprzez wykonanie komendy.
```
sudo apt install memcached
```

Teraz zainstaluje klient do memcached dla języka Python
```
pip install pymemcache
```

Musimy uruchomić serwer z bazą, najlepiej w osobnym terminalu.
```
memcached
```

Następnie możemy zacząć pisać program wykorzystujący naszą bazę w Pythonie.

In [ ]:
from pymemcache.client import base

# Tworzymy obiekt klienta, podająć localhost oraz domyślny port memcached czyli 11211
client = base.Client(('localhost', 11211))

# Ustawiamy wartość z kluczem klucz oraz wartość
client.set(key="2+2", value=4)

# Teraz możemy odzyskać wartość pod tym kluczem
print(client.get(key="2+2"))

b'4'


Powyższe użycie jest tak naprawdę całą ideą takiej bazy. Teraz każdy serwer w naszej sieci może zapytać się na porcie 11211 o to czy wiemy jaka jest wartość dla klucza "2+2" i dostanie odpowiedź.

Klucze w memcached mają określony czas ważności, po którym znikają. (aby zapobiec przepełnieniu pamięci). Innym mechanizmem jest też wyrzucanie aktualnych danych (według różnych algorytmów np. LRU)

Gdy spróbujemy uzyskać klucz który nie istnieje w bazie, nie otrzymamy żadnej wartości. A dokładnie None (w pythonie obiekt oznaczający brak wartości).

In [ ]:
# Szukamy wartości pod kluczem który nie istnieje w bazie
value = client.get(key="1+1")

# Otrzymaliśmy None, oznacza to że nie znaleziono żadnej wartości
print(value)

None


Możemy również ustawić wartość domyślną, która ma być zwracana gdy nie znajdziemy wartości.

In [ ]:
# Ustawiamy argument default jako "Player", taką wartość otrzymamy jeśli nie zostanie znaleziony klucz o tej nazwie w bazie.
value = client.get(key="player_name", default="Player")

print(value)

Player


A teraz praktyczne użycie, chcemy uzyskać jakąś informację z API

In [ ]:
def get_temperature(city: str) -> int:
    # tutaj pytamy api, jednak dla przykładu zwracamy wartość 4
    return 4

response = client.get(key="Cracow")

if response is None:
    # Jesli nie istnieje klucz w bazie, robimy query i zapisujemy odpowiedź
    response = get_temperature(city="Cracow")

    # Dodajemy tą wartość do bazy danych
    client.set(key="Cracow", value=response)

print(response)

b'4'


Możemy też używać bazy jako licznik wykorzystując polecenia incr i decr

In [ ]:
client.set("Visitors", 0)
print(client.get("Visitors"))

client.incr("Visitors", 10)
print(client.get("Visitors"))

client.decr("Visitors", 2)
print(client.get("Visitors"))

b'0'
b'10'
b'8 '


Możemy zadać pytanie po co takie funkcje jak powyżej, możemy przecież zrobić implementację jak poniżej. 

Jest jednak z tym problem, zzy jesteś w stanie go dostrzec? (Przypominam że memcached jest wielowątkowy)

In [ ]:
value = client.get("Visitors")
if not value:
    value = 1
else:
    value += 1

client.set("Visitors", value)

Opis problemu znajduję sie poniżej.

Problemem jest atomowość operacji, jako że memcached jest wielowątkowy i rozproszony, może być podłączonych do niego wielu klientów jednocześnie.

Sprawa to problem gdy jednocześnie wielu klientów zmieni value w powyższy sposób, stanie się tak zwane race condition i wartość może zostać nieuwzględniona. Operacje incr i decr zapobiegają temu.

Dla wartości nie intowych gdzie incr i decr nie zadziała możemy użyć innego mechanizmu, gdzie zwracany przy set jest specjalny obiekt który potem przekazujemy w get, jest to jednak bardziej skomplikowane i pominę to w tej prezentacji. (Informacje o tym można znaleźć pod nazwą "memcached cas")

#### Podsumowanie

Memcached jest prostym w użyciu, lecz potężnym narzędziem gdy możemy zapisywać odpowiedzi, przydatne gdy np. tworzymy jakis serwis gdzie dane są bardzo często odczytywane.

---

### Redis

Przejdziemy teraz do innej bazy danych typu in-memory, jest ona jednak bardziej skomplikowana. Idea jednak jest podobna, Redis też może zostać użyty jako cache, ma jednak znacznie więcej zastosowań takie jak np .kolejkowanie tasków (np. do asynchronicznego programowania). Jednak skupię się na użyciu go jako typowa baza danych.

Może on przechowywać znacznie większe struktury niż memcached, aż do 512MB. Wspiera typy typu listy, zbiory czy nawet typ danych geolokalizacyjnych (geohash).

Jednak ma on jedną istotną cechę różniącą go od memcached, ma możliwość zapisania danych na dysku a więc zachowania "Persistence" nawet po wyłączeniu komputera.
Może osiągnąć to na dwa różne sposoby:

- Snapshotting - czyli zapisywanie danych co jakiś określony czas, np. co sekundę lub co 10 sekund, minutę...  Np. dla snapshota co sekundę przy awarii systemu stracimy tylko sekundę danych, przy 10 stracimy 10 sekund itd. Im niższy czas tym oczywiście większy spadek wydajności.
- Journaling - Z każdą operacją dodawany jest journal log (zapis operacji) do pliku zooptymalizowanego pod append-only, czyli pod zapis na koniec. Ten tryb pozwala na zapisanie wszystkich danych jednak ma znacznie większy koszt wydajnościowy (koszt zapisu z każdą operacją). Również plik może stać się bardzo duży.